In [10]:
#어제에 이어서...
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from konlpy.tag import Komoran

ma = Komoran()

stopwords == '은 는 이 가 을 을 에게 께서'.split()
#한국어는 우리가 직접 만들어줘야한다. 

data = '''
어머니는 자장면이 싫다고 하셨어.
엄마는 짜장면이 넘나 싫다고 했어.
우리 엄마는 짜장이 극혐이라 했었어.
'''
#여기서 실질적 표혀으는 <어머니 짜장 싫다> 임

#문장 분리해볼거임.
for s in sent_tokenize(data):
    #형태소 분리를 해볼거임.
    print(ma.morphs(s))

['어머니', '는', '자장면', '이', '싫', '다고', '하', '시', '었', '어', '.']
['엄마', '는', '짜장면', '이', '넘', '나', '싫', '다고', '하', '았', '어', '.']
['우리', '엄마', '는', '짜', '장이', '극', '혀', 'ㅁ', '이', '라', '하', '았었', '어', '.']


In [11]:
#근데 유니크한 단어들만 모아볼거임.
unique = list()
for s in sent_tokenize(data):
    unique.extend(ma.morphs(s))
len(list(set(unique)))
#필요없는 애들 넘 많아서 

24

In [12]:
unique = list()
for s in sent_tokenize(data):
    unique.extend(ma.morphs(s))
    print(ma.morphs(s))

['어머니', '는', '자장면', '이', '싫', '다고', '하', '시', '었', '어', '.']
['엄마', '는', '짜장면', '이', '넘', '나', '싫', '다고', '하', '았', '어', '.']
['우리', '엄마', '는', '짜', '장이', '극', '혀', 'ㅁ', '이', '라', '하', '았었', '어', '.']


In [13]:
#특징들이 있으니, 코모란 기준으로 보는거임. 모아볼거임. 태그 달아서
ma.tagset['JX'], ma.tagset['JKS'], ma.tagset['EC'], ma.tagset['EP']
#이런 애들이 많이 붙어있는걸 볼 수 있다. 

('보조사', '주격 조사', '연결 어미', '선어말어미')

In [17]:
#무슨 얘기를 하고 싶은거냐면, 
unique = list()
pos = list()
for s in sent_tokenize(data):
    unique.extend(ma.morphs(s))
    pos.extend([t[1][0]for t in ma.pos(s)])

In [18]:
from collections import Counter
sum(Counter(pos).values()), Counter(pos)

(37, Counter({'E': 12, 'N': 9, 'V': 8, 'J': 5, 'S': 3}))

In [21]:
unique = list()
pos = list()
for s in sent_tokenize(data):
    unique.extend(ma.morphs(s))
    pos.extend([t[1][0]for t in ma.pos(s)])
    #의미가 전달할때 전달되는지 보는거임...
    print(' '.join([t[0] for t in ma.pos(s) 
                    if t[1][0] not in ['J', 'E']]))
#형식형태소를 제거해도 원래 표현하려는 의미에서 크게 달라지지 않았다. 

어머니 자장면 싫 하 .
엄마 짜장면 넘 싫 하 .
우리 엄마 짜 장이 극 혀 이 하 .


In [26]:
#형태소 이용한다고 했을때,
unique = list()
pos = list()
for s in sent_tokenize(data):
    print(' '.join([t for t in ma.morphs(s) 
                    if t not in stopwords and \
                    len(t) > 1]))

#왜 에러나지....

TypeError: argument of type 'WordListCorpusReader' is not iterable

##### 1음절짜리들이 의미가 있다면?
한 사람
1   2
수사에 해당하는 애들은 필요한 애일거다. 이런애들은 N-gram 을 섞어서 쓰면 된다. 

N-gram 쓰면 

    '한 ', ' 한'
    
    이렇게 나온다함.... 1음절짜리를 피할수있다고 함..

## 욕설제거

In [27]:
from nltk.tokenize import word_tokenize

stopwords = ['시발']
#누군가 input 창에 친다.
s = input()
#input 내용을 토크나이징을 할거다.  word _tokenize 할거임.
' '.join([t if t not in stopwords else '*'*len(t)
          for t in word_tokenize(s)])


 어쩌고 저쩌고 시발


'어쩌고 저쩌고 **'

In [28]:
#시발이 막히면, 다른 형태로 욕을 할거임.
s = input()
' '.join([t if t not in stopwords else '*'*len(t)
          for t in word_tokenize(s)])

 씨발


'씨발'

In [30]:
#그러면, 욕에 대한 경우를 좀 더 추가해준다. 
#그러면, 또 막아줘야 한다.
import re
from nltk.tokenize import word_tokenize
from string import punctuation

stopwords = ['시발', '씨발']
#욕설을 제거하는 정규식 패턴
pl = re.compile(f'[{re.escape(punctuation)}]')

s = input()
' '.join([t if t not in stopwords else '*'*len(t)
          for t in word_tokenize(pl.sub('', s))])

 씨!!발!!


'**'

In [31]:
#욕설이 막아졌다! 

In [37]:
#근데 이런 주먹구구식으로 해결해야할까?
import re
from nltk.tokenize import word_tokenize
from string import punctuation

stopwords = ['시발', '씨발']
pl = re.compile(f'[{re.escape(punctuation)}]')
#패턴을 하나더 만들어줬다.
# p2 = re.compile(r'(?:\s|\b)\W(?:\s|\b)')
p2 = re.compile(r'\B\W\B') #위와 같은 의미임

s = input()
' '.join([t if t not in stopwords else '*'*len(t)
          for t in word_tokenize(p2.sub(' ', pl.sub('', s)))])

 시 발


'시 발'

In [38]:
#그래도 시이발이라고 쓰면.. 안걸러진다..ㅠ
# 사람이 직접 입력하는게 아니라 시|씨|ㅆ(.*)발\b 이렇게 데이터로부터 자동으로 확인해서 올 수 있을까?
# 가능하다고 한다. ... 우리가 지금까지 배운 방식 중에서.. 

In [83]:
#BPE =>
data = '''
시발 시발 씨발 시이발 시~@#@%발 시부렁
'''

data = Counter(data.split())

stopwords = dict()
for k, v in data.items():
    stopwords[('<w>',) + tuple(k)+('<w>',)] = v #['</w>'] 이걸 쪼개는 이유는..? 낱개의 스필릿으로 조개지 않기 위해..?

stopwords

{('<w>', '시', '발', '<w>'): 2,
 ('<w>', '씨', '발', '<w>'): 1,
 ('<w>', '시', '이', '발', '<w>'): 1,
 ('<w>', '시', '~', '@', '#', '@', '%', '발', '<w>'): 1,
 ('<w>', '시', '부', '렁', '<w>'): 1}

In [84]:
data

Counter({'시발': 2, '씨발': 1, '시이발': 1, '시~@#@%발': 1, '시부렁': 1})

In [85]:
from collections import defaultdict
#defaultdict란?
# default 값이 있는거다. 원래 key에 value 값이 없거나 하면, 에러가 나는데, 
defaultdict(int)['key']
#없으면, insert 하면서 default 값으로 보여주기 때문에. 

0

In [86]:
from collections import defaultdict

def findPair(kv):
    pairs = defaultdict(lambda:0)

    for k, v in kv.items():
        for i in range(len(k)-1):
            #이 구문을 그냥 쓰면 에러가 나는데 defaultdict 때메 안나는거임
            pairs[' '.join(k[i:i+2])] += v

In [87]:
p = findPair(stopwords)
max(p, key=p.get)

AttributeError: 'NoneType' object has no attribute 'get'

In [88]:
keys = list()
for _ in range(3):
    pairs = findPair(stopwords)
    maxkey = max(pairs, key=pairs.get)
    keys.append(maxkey)
    temp = dict()
    
    #합치는 부분이다.
    for k,v in stopwords.items():
        newkey = tuple(re.sub(maxkey, maxkey.replace(' ',''),
                              ' '.join(k)).split())
        temp[nk] = v
    stopwords = temp

AttributeError: 'NoneType' object has no attribute 'get'

In [89]:
stopwords

{('<w>', '시', '발', '<w>'): 2,
 ('<w>', '씨', '발', '<w>'): 1,
 ('<w>', '시', '이', '발', '<w>'): 1,
 ('<w>', '시', '~', '@', '#', '@', '%', '발', '<w>'): 1,
 ('<w>', '시', '부', '렁', '<w>'): 1}

In [90]:
keys

[]

In [91]:
#단어를 뒤질거다.. 찾은 키들에 대해서.. map 할 거임..<w>를 만나면, word boundary 로 바꿔라는거. 
# list(map(lambda k:re.sub(r'</?w>', r'\b', k), keys))
# 이 코드 그냥 안씀 .. ... 

[]

In [97]:
#다시 쓰기
list(map(lambda k:re.sub('</?w>', r'\b', k), keys))

[]

#### 1케이스: <w>글자, 2케이스: 글자</w>, 3케이스: <w>글 자</w> 를 만나면
앞뒤 어느 한쪽이 \b 아니면, 1 or 2, 1+2 이것들의 조합으로 마치
<w>시발</w>, <w>씨발</w>
이걸 알아서 조합해서 하는거다. 

In [99]:
#슈도 코드의 의도는 저러한데, 자꾸안되서 쌤이 집가서 찾아보고 알아오겠다함.

## 우리말

In [100]:
ord('가'), ord('나')

(44032, 45208)

In [101]:
ord('나') - ord('가') 
#이 만큼의 차이가 있다.

1176

In [102]:
ord('갸') - ord('가') 

56

In [103]:
ord('각') - ord('가') 

1

In [104]:
#종성이 붙어있는 형태라는 걸 알 수 있다.

In [111]:
#가 의 형태로 50글자를 뽑아내보면?
print([chr(ord('가')+i) for i in range(50)])

['가', '각', '갂', '갃', '간', '갅', '갆', '갇', '갈', '갉', '갊', '갋', '갌', '갍', '갎', '갏', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '갘', '같', '갚', '갛', '개', '객', '갞', '갟', '갠', '갡', '갢', '갣', '갤', '갥', '갦', '갧', '갨', '갩', '갪', '갫', '갬', '갭', '갮', '갯', '갰', '갱']


In [110]:
#중성 종성 28
print([chr(ord('가')+i) for i in range(28)])

['가', '각', '갂', '갃', '간', '갅', '갆', '갇', '갈', '갉', '갊', '갋', '갌', '갍', '갎', '갏', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '갘', '같', '갚', '갛']


In [109]:
print([chr(ord('ㄱ')+i) for i in range(30)])

['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


In [112]:
#종성에 안들어가는것 부터 걸러내자. 격자어 ㅃ ㄸ 이런거 안들어감. 

In [115]:
#종성 만들어보기. 종성에 해당되는 애들은 ㄲ, ㄸ, ㅉ 빼고 받침 없는 28자.
jong = [' ', 'ㄱ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',  'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ',  'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

In [116]:
len(jong)

28

In [117]:
print([chr(ord('가')+(28*i)) for i in range(21)])
#중성에 해당하는 모음이 21자다. 

['가', '개', '갸', '걔', '거', '게', '겨', '계', '고', '과', '괘', '괴', '교', '구', '궈', '궤', '귀', '규', '그', '긔', '기']


In [118]:
#하나의 초성으로 시작하는게 588 반복? 아 모르겠다...

In [119]:
# jung = ['가', '개', '갸', '걔', '거', '게', '겨', '계', '고', '과', '괘', '괴', '교', '구', '궈', '궤', '귀', '규', '그', '긔', '기']
jung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', '괴', '교', '구', '궈', '궤', '귀', '규', '그', '긔', '기']

In [126]:
#이제 초성만 찾으면 된다. 588자마다..
print([chr(ord('가')+(588*i)) for i in range(19)])

['가', '까', '나', '다', '따', '라', '마', '바', '빠', '사', '싸', '아', '자', '짜', '차', '카', '타', '파', '하']


In [127]:
cho = ['ㄱ', 'ㄲ', 'ㄴ',  'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ',  'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
len(cho)

19

In [134]:
len(cho), len(jung), len(jong), 19*21*28

(19, 21, 28, 11172)

In [157]:
#음절을 바꾸는 함수
def chr2tri(c):
    if re.match('[가-힣]', c):
        #가 를 계속 써야하니 , 
        base = divmod(ord(c)-ord('가'), (21*28))
        #처음에 초성 쪼개기 (중성과종성의 모든 조합으로 나누기)
        c1 = base[0]
        c2 = divmod(base[-1], 28)[0]
        c3 = divmod(base[-1], 28)[-1]
        return cho[c1]+jung[c2]+jong[c3]
    else:
        return c
        
def tri2char(c):
    #3음절 가정
    if re.match(r'^[ ㄱ-ㅎㅏ-ㅣ]{3}$', c):
        r = ord('가') + cho.index(c[0])*21*28
        r += jung.index(c[1])*28
        r += jong.index(c[2])
        return chr(r)
    else:
        return c
result = tri2char('ㄱㅏ '), tri2char(chr2tri('걁'))
print(result)

ValueError: 'ㅏ' is not in list

In [143]:
divmod(40,2)[0] /2

10.0

### 형태소 => 1음절 단어 => 초중종

In [158]:
ma.pos('안녕하세요'), ma.pos('아녕하세요')

([('안녕하세요', 'NNP')], [('아녕하세요', 'NA')])

In [159]:
ma.tagset['NA']

'분석불능범주'

## Edit Distance
Hamming Distance

In [160]:
def hamming(s1, s2):
    if len(s1) != len(s2) :
        return '분석불가'
        
    return sum([c1 != c2 for c1, c2 in zip(s1, s2)])
hamming('안녕하세요', '안ㄴ녕하세')        

4

In [161]:
#그래서 나오는게 Levenshtein Distance
def leven(s1, s2):
    if len(s1) == 0:
        return len(s2)
    if len(s2) == 0:
        return len(s1)
    if s1[0] == s2[0]:
        return leven(s1[1:], s2[1:])
    else:
        #가중치를 매기자.
        v1 = 1 + leven(s1[1:], s2[1:]) #교체
        v2 = 1 + leven(s1, s2[1:]) #추가
        v3 = 1 + leven(s1[1:], s2) #삭제
        return min((v1, v2, v3))
        
leven('', 'abd'), leven('abc', '')

(3, 3)

In [164]:
leven('안녕하세요', '아녕하세요'),leven('고려대', '고구려대')

(1, 1)

In [167]:
#불공평해서 더 패널티를 주겠다.
def leven(s1, s2):
    if len(s1) == 0:
        return len(s2)
    if len(s2) == 0:
        return len(s1)
    if s1[0] == s2[0]:
        return leven(s1[1:], s2[1:])
    else:
        
        v1 = .1 + leven(s1[1:], s2[1:]) # 오타난 경우는 불쌍하니까, 패널티를 적게 주자. 
        v2 = 3 + leven(s1, s2[1:]) #추가된 글자는 패널티 많이
        v3 = 2 + leven(s1[1:], s2) #삭제된 글자의 패널티
        return min((v1, v2, v3))

leven('안녕하세요', '아녕하세요'),leven('고려대', '고구려대')

#이걸로 검색창에 오타 쳐도 알아서 '~~~'로 검색했지 않냐고, 알아서 수정해서 검색해주는데 쓰일 수 있다. 

(0.1, 1.2000000000000002)

In [168]:
leven('안녕하세요', '아녕하세요'),leven('고려대', '고신대')
#근데 이것도 억울하다.
#그래서 초성중성을 쪼개는거다.

(0.1, 0.1)

In [169]:
#초성, 중성을 쪼개보자
s1 = ''.join(list(map(lambda c:chr2tri(c), '안녕하세요')))
s2 = ''.join(list(map(lambda c:chr2tri(c), '아녕하세요')))
leven(s1, s2)

0.1

In [170]:
s1 = ''.join(list(map(lambda c:chr2tri(c), '한성대')))
s2 = ''.join(list(map(lambda c:chr2tri(c), '한세대')))
leven(s1, s2)
#차이를 더 벌리고 싶으면 종성에 대한 처리를 해주면 된다. 

0.2